In [115]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from model import Model
import utils

# Getting data
For this task we will use an sample salary data.

In [116]:
n_train, n_test = 1000, 200
n_inputs, batch_size = 200, 10

weights, bias = utils.get_weights_and_bias(n_inputs)

train_data = utils.create_data(weights, bias, n_train)
test_data = utils.create_data(weights, bias, n_test)

SyntaxError: invalid syntax (3873064544.py, line 4)